Задание:

На основе Правил безопасности опасных производственных объектов, на которых используются подъемные сооружения (Документ для создания базы знаний: https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc), сделайте:
- нейро-консультанта для ответов на вопросы по представленному документу, проработайте промт самостоятельно;
- Проверьте работоспособность созданного консультанта на нескольких самостоятельно сформулированных вопросах.

In [2]:
!pip install -q faiss-cpu==1.8.0 mistralai tiktoken==0.7.0 langchain==0.2.14 langchain_openai==0.1.22 langchain_community==0.2.12 sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00


In [17]:
# запустите эту ячейку, если используете секретный ключ в колабе
import os
from mistralai import Mistral
from google.colab import userdata
api_key = userdata.get("MISTRAL_API_KEY")
os.environ["MISTRAL_API_KEY"] = api_key

In [4]:
# запустите эту ячейку, если используете секретный ключ в колабе

import os
from google.colab import userdata
# Получение API ключа из пользовательских данных Colab и установка его как переменной среды
key = userdata.get('MISTRAL_API_KEY')
os.environ["MISTRAL_API_KEY"] = key

In [5]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
import openai
from langchain.docstore.document import Document

In [6]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text


In [7]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc')

In [11]:
# деление на чанки
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={"meta":"data"}))
print("Общее количество чанков: ", len(source_chunks))

Общее количество чанков:  238


In [13]:
# Инициализирум модель эмбеддингов
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Функция для создания эмбеддингов из массива документов
def create_embeddings(documents):
    page_contents = [doc.page_content for doc in documents]
    return model.encode(page_contents)

# Создание эмбеддингов
embeddings = create_embeddings(source_chunks)

# Создадим индексную базу из разделенных фрагментов текста
import numpy as np
import faiss
# Преобразование эмбеддингов в массив numpy
embeddings = np.array(embeddings).astype('float32')

# Создание индекса FAISS
dimension = embeddings.shape[1]  # Размерность векторов
index = faiss.IndexFlatL2(dimension)  # Использование L2 метрики

# Добавление векторов в индекс
index.add(embeddings)


In [18]:
# основная функция для получения ответов по БЗ
def answer_index(system, topic, query_vector, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    distances, indices =  search_index.search(query_vector, k=4)
    #indices_list = indices[0].tolist()
    # Вывод ближайших документов
    docs = [source_chunks[i] for i in indices[0]]

    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = Mistral(api_key=api_key)
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос сотрудника компании. Не упоминай документ с информацией для ответа сотруднику в ответе. Документ с информацией для ответа сотруднику: {message_content}\n\nВопрос сотрудника: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.complete(model="mistral-small-latest", messages=messages)
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [15]:
# самостоятельно напишите промпт
system="Вы — нейроконсультант, специализирующийся на правилах безопасности опасных производственных объектов, связанных с использованием подъемных сооружений. Ваша задача — отвечать на вопросы пользователей, основываясь на документе. Не придумывай ничего от себя, отвечай максимально по документу. Не упоминай Документ с информацией для ответа клиенту. Клиент ничего не должен знать про Документ с информацией для ответа клиенту. При необходимости приводите примеры и рекомендации, основанные на правилах безопасности, чтобы помочь пользователям лучше понять материал. "

In [19]:
# проверьте работу нейро-консультанта на нескольких самостоятельно сформулированных вопросах, для получения более качественных ответов
# можно поэкспериментировать с разделением на чанки и другими параметрами
topic= "Что необходимо проверить перед началом эксплуатации подъемного сооружения?"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')
answer = answer_index(system, topic, query_vector, index)
answer


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================наводнений, землетрясений, селей, произошедших на территории нахождения ПС. Требования к процессу эксплуатации, проверке состояния и дефектации грузозахватных приспособлений и тары 215. Требования промышленной безопасности при эксплуатации грузозахватных приспособлений, в том числе к проведению технического обслуживания, ремонта, реконструкции, должны соответствовать требованиям промышленной безопасности при эксплуатации ПС, совместно с которым они используются по назначению. 216. Персонал, который назначается для выполнения работ по строповке, в том числе, по навешиванию на крюк ПС, зацепке и обвязке грузов, перемещаемых ПС с применением грузозахватных приспособлений, должен иметь уровень квалификации, соответствующий профессии "стропальщик". То же требование предъявляется к персоналу основных рабочих профессий, в обязанности которых входит подвешивание на крюк груза без предв

'Перед началом эксплуатации подъемного сооружения необходимо провести комплексный осмотр для обеспечения его безопасности. В частности, следует проверить:\n\n1. **Состояние конструкции**: Убедитесь, что все элементы подъемного сооружения находятся в хорошем состоянии и не имеют видимых повреждений, таких как трещины, деформации или коррозия.\n\n2. **Функционирование механизмов**: Проверьте работу всех механизмов, таких как подъемники, перемещающие устройства и блокировки. Убедитесь, что они работают гладко и без сбоев.\n\n3. **Электро-, пневмо- или гидравлический привод**: Если подъемное сооружение имеет электро-, пневмо- или гидравлический привод, проверьте их работоспособность и соответствие требованиям безопасности.\n\n4. **Указатели, ограничители и регистраторы**: Убедитесь, что все регистраторы, ограничители и указатели находятся на своих местах и работают корректно.\n\n5. **Система управления**: Проверьте наладку системы управления подъемным сооружением в целом. Убедитесь, что вс

In [20]:
# вопрос пользователя
topic= "Как действовать работнику, если он заметил неисправность подъемного сооружения во время работы?"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')
answer = answer_index(system, topic, query_vector, index)
answer


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================отрегулированы согласно руководству по эксплуатации на тормозной момент, указанный в паспорте ПС, а ограничитель грузоподъемности - отключен. Статические испытания следует проводить для каждого грузоподъемного механизма и, если это предусмотрено в паспорте ПС, при совместной работе грузоподъемных механизмов в положениях и вариантах исполнения, выбранных таким образом, чтобы усилия в канатах, изгибающие моменты и (или) осевые усилия в основных элементах ПС были наибольшими. Статические испытания должны проводиться со следующими нагрузками (по отношению к номинальной паспортной грузоподъемности): 125 процентов - для ПС всех типов (кроме подъемников); 140 процентов - для кранов-трубоукладчиков; 200 процентов - для грузопассажирских и фасадных строительных подъемников; 150 процентов - для грузовых строительных подъемников (при невыдвинутом грузонесущем устройстве); 125 процентов - 

'Если работник заметил неисправность подъемного сооружения во время работы, ему следует предпринять следующие действия:\n\n1. **Остановить работу**: Немедленно прекратить выполнение работы и отключить подъемное сооружение.\n2. **Уведомить руководство**: Оповестить о неисправности непосредственного руководителя или ответственного лица.\n3. **Обеспечить безопасность**: Убедиться, что все работники находятся в безопасности и не находятся под угрозой.\n4. **Документировать неисправность**: Записать все наблюдаемые неисправности и обстоятельства, при которых они были обнаружены.\n5. **Не пытаться самостоятельно устранить неисправность**: Работник не должен предпринимать попытки самостоятельно исправить неисправность, если он не обладает соответствующими навыками и полномочиями.\n\nЭти действия помогут предотвратить возможные аварийные ситуации и обеспечат своевременное устранение неисправности.'

In [21]:
# вопрос пользователя
topic= "Что делать в случае аварии или инцидента с подъемным сооружением?"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')
answer = answer_index(system, topic, query_vector, index)
answer


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================этом крановые, крано-манипуляторные установки и подъемные установки подъемников (вышек) стационарно закреплены на шасси или раме. 3. Требования настоящих ФНП не распространяются на обеспечение безопасности объектов, на которых используются следующие ПС: а) применяемые в интересах обороны и безопасности государства, гражданской и территориальной обороны или относящиеся к вооружению и военной технике, кроме ПС общепромышленного назначения, перечисленных в пункте 3 настоящих ФНП и предназначенных только для транспортировки обычных грузов, и кроме ПС, применяемых на ОПО, эксплуатируемых организациями Государственной корпорации по атомной энергии "Росатом" (далее - Госкорпорация "Росатом"), при разработке, изготовлении, испытании, эксплуатации и утилизации ядерного оружия и ядерных установок военного назначения; б) специально сконструированные для применения на вводимых в эксплуатац

'В случае аварии или инцидента с подъемным сооружением необходимо предпринять следующие меры:\n\n1. **Остановить работу**: Немедленно прекратить работу подъемного сооружения и отключить его от источника питания.\n2. **Обеспечить безопасность**: Убедиться, что все работники находятся в безопасном месте и не подвергаются риску.\n3. **Уведомить начальство**: Немедленно сообщить о происшествии руководству или ответственному лицу в организации.\n4. **Оказать первую помощь**: В случае наличия пострадавших, оказать им первую помощь и вызвать скорую медицинскую помощь.\n5. **Изолировать зону происшествия**: Запретить доступ к месту инцидента до проведения расследования и устранения неполадок.\n6. **Провести расследование**: Определить причины инцидента и принять меры по предотвращению подобных ситуаций в будущем.\n7. **Задокументировать происшествие**: Сделать запись о происшествии, включая временные данные, описание событий, имена свидетелей и пострадавших.\n8. **Провести техническую эксперти

In [22]:
# вопрос пользователя
topic= "Каковы требования к квалификации работников, осуществляющих монтаж и ремонт подъемных сооружений?"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')
answer = answer_index(system, topic, query_vector, index)
answer


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================этом крановые, крано-манипуляторные установки и подъемные установки подъемников (вышек) стационарно закреплены на шасси или раме. 3. Требования настоящих ФНП не распространяются на обеспечение безопасности объектов, на которых используются следующие ПС: а) применяемые в интересах обороны и безопасности государства, гражданской и территориальной обороны или относящиеся к вооружению и военной технике, кроме ПС общепромышленного назначения, перечисленных в пункте 3 настоящих ФНП и предназначенных только для транспортировки обычных грузов, и кроме ПС, применяемых на ОПО, эксплуатируемых организациями Государственной корпорации по атомной энергии "Росатом" (далее - Госкорпорация "Росатом"), при разработке, изготовлении, испытании, эксплуатации и утилизации ядерного оружия и ядерных установок военного назначения; б) специально сконструированные для применения на вводимых в эксплуатац

'Требования к квалификации работников, осуществляющих монтаж и ремонт подъемных сооружений, включают следующие аспекты:\n\n1. **Обучение и сертификация**: Работники должны пройти специальное обучение и получить сертификаты, подтверждающие их квалификацию. Это может включать курсы по технике безопасности, правилам эксплуатации и техническому обслуживанию подъемных сооружений.\n\n2. **Должностные инструкции**: Инженерно-технические работники должны иметь должностные инструкции, которые определяют их обязанности и ответственность при выполнении монтажных и ремонтных работ.\n\n3. **Производственные инструкции**: Персонал должен иметь производственные инструкции, которые включают детализированные процедуры и требования по безопасности при выполнении работ.\n\n4. **Опыт и практика**: Работники должны иметь соответствующий опыт и практику в области монтажа и ремонта подъемных сооружений. Это может включать участие в предыдущих проектах и выполнение аналогичных работ.\n\n5. **Соблюдение нормат